<a href="https://colab.research.google.com/github/nahumsa/DM-Reconstruction/blob/Tensorflow_test/Trace%20Distance%20Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [83]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
print(tf.__version__)

2.2.0-rc3


# Tests

In [0]:
#Creating pauli matrices
sigma_0_np = np.array([[1.,0.],
                      [0.,1.]], dtype=np.complex64)

sigma_1_np = np.array([[0.,1.],
                       [1.,0.]], dtype=np.complex64)

sigma_2_np = np.array([[0.,1.j],
                       [-1.j,0.]], dtype=np.complex64)

sigma_3_np = np.array([[1.,0.],
                       [0.,-1.]], dtype=np.complex64)

#Converting to tensors
sigma_0 = tf.Variable(sigma_0_np, tf.complex64)

sigma_1 = tf.Variable(sigma_1_np, tf.complex64)

sigma_2 = tf.Variable(sigma_2_np, tf.complex64)

sigma_3 = tf.Variable(sigma_3_np, dtype=tf.complex64)

In [90]:
meas = [[1,1,0],[0,0,1],[0,1,1],[0,1,1],[.4,1,1]]
measurement = np.array(meas, dtype=np.complex64)
measurement_tensor = tf.Variable(measurement, tf.complex64)
print(measurement_tensor)

<tf.Variable 'Variable:0' shape=(5, 3) dtype=complex64, numpy=
array([[1. +0.j, 1. +0.j, 0. +0.j],
       [0. +0.j, 0. +0.j, 1. +0.j],
       [0. +0.j, 1. +0.j, 1. +0.j],
       [0. +0.j, 1. +0.j, 1. +0.j],
       [0.4+0.j, 1. +0.j, 1. +0.j]], dtype=complex64)>


In [0]:
basis = tf.Variable([sigma_1,sigma_2,sigma_3])

In [92]:
density_matrix = 0.5*(sigma_0 + tf.tensordot(measurement_tensor,basis,axes=1))
print(density_matrix)

tf.Tensor(
[[[0.5+0.j  0.5+0.5j]
  [0.5-0.5j 0.5+0.j ]]

 [[1. +0.j  0. +0.j ]
  [0. +0.j  0. +0.j ]]

 [[1. +0.j  0. +0.5j]
  [0. -0.5j 0. +0.j ]]

 [[1. +0.j  0. +0.5j]
  [0. -0.5j 0. +0.j ]]

 [[1. +0.j  0.2+0.5j]
  [0.2-0.5j 0. +0.j ]]], shape=(5, 2, 2), dtype=complex64)


In [95]:
tf.linalg.eigvalsh(density_matrix)

<tf.Tensor: shape=(5, 2), dtype=complex64, numpy=
array([[-0.20710672+0.j,  1.2071067 +0.j],
       [ 0.        +0.j,  1.        +0.j],
       [-0.20710677+0.j,  1.2071067 +0.j],
       [-0.20710677+0.j,  1.2071067 +0.j],
       [-0.2348469 +0.j,  1.234847  +0.j]], dtype=complex64)>

# Methods

In [0]:
def create_density_mat(measurements: np.array) -> tf.Tensor:
  
  #Creating the basis
  sigma_0_np = np.array([[1.,0.],
                        [0.,1.]], dtype=np.complex64)

  sigma_1_np = np.array([[0.,1.],
                         [1.,0.]], dtype=np.complex64)

  sigma_2_np = np.array([[0.,1.j],
                         [-1.j,0.]], dtype=np.complex64)

  sigma_3_np = np.array([[1.,0.],
                        [0.,-1.]], dtype=np.complex64)

  #Converting to tensors
  sigma_0 = tf.Variable(sigma_0_np, tf.complex64)

  sigma_1 = tf.Variable(sigma_1_np, tf.complex64)

  sigma_2 = tf.Variable(sigma_2_np, tf.complex64)

  sigma_3 = tf.Variable(sigma_3_np, dtype=tf.complex64)

  basis = tf.Variable([sigma_1,sigma_2,sigma_3])
  
  measurement = np.array(measurements, dtype=np.complex64)
  measurement_tensor = tf.Variable(measurement, tf.complex64)

  density_matrix = 0.5*(sigma_0 + tf.tensordot(measurement_tensor,basis,axes=1))
  return density_matrix


In [0]:
def trace_dist(A,B):
  dif = A - B
  dif = tf.transpose(dif, conjugate=True, perm=[0,2,1]) * dif  
  vals = tf.linalg.eigvalsh(dif)
  return tf.math.real(0.5*tf.reduce_sum(tf.math.sqrt(tf.math.abs(vals)),axis=-1))

In [0]:
def trace_loss(y_true,y_pred):
  d_y_true = create_density_mat(y_true)  
  d_y_pred = create_density_mat(y_pred)  
  return tf.reduce_mean(trace_dist(d_y_pred,d_y_true))

In [139]:
y_1 = [[.5,.5,0],[.2,0.3,.3]]
y_2 = [[1,0,0],[.2,.3,.3]]
trace_loss(y_true=y_1, y_pred=y_2)

<tf.Tensor: shape=(), dtype=float32, numpy=0.17677669>